In [51]:
import pandas as pd
import json as js

with open("/u/pmpande/author_profiling/essays.json") as f:
    data_essays = pd.DataFrame(js.loads(line) for line in f)
f.close()

print(len(data_essays))

2469


In [52]:
data_essays.drop(data_essays.head(1).index, inplace=True)

In [ ]:
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import tree
from empath import Empath
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from math import sqrt
from sklearn import metrics
from sklearn.preprocessing import normalize

accuracy = 0.0
rmse = 0.0
h_loss = 0.0
for i in range(10):
    rand_essays = []
    temp_data = data_essays.copy()
    rand_essays = random.sample(range(1,len(temp_data)-1),250)
    test = temp_data.iloc[rand_essays].copy()
    train = temp_data.drop(rand_essays)
    train.drop(train.columns[[1]], axis=1, inplace=True)
    test.drop(test.columns[[1]], axis=1, inplace=True)
    
    corpus = []
    truth = []
    for row in train.iterrows():
        Y = [0,0,0,0,0]
        corpus.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth.append(Y)
    truth = np.array(truth)    
    lexicon = Empath()
    feat = []
    a = {}
    for row in corpus:
        a = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a.items():
            temp.append(v)
        feat.append(temp)
    feat = np.array(feat)
    
    feat_norm = normalize(feat, axis=1, norm='l1')
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(feat_norm, truth)
    
    test_data = []
    truth_test = []
    for row in test.iterrows():
        Y = [0,0,0,0,0]
        test_data.append(row[1]['STATUS'])
        if row[1]['AGR'] == 'y': Y[0] = 1
        if row[1]['CON'] == 'y': Y[1] = 1
        if row[1]['EXT'] == 'y': Y[2] = 1
        if row[1]['NEU'] == 'y': Y[3] = 1
        if row[1]['OPN'] == 'y': Y[4] = 1
        truth_test.append(Y)
    truth_test = np.array(truth_test)    
    test_feat = []
    a_test = {}
    for row in test_data:
        a_test = lexicon.analyze(row, normalize=True)
        temp = []
        for k,v in a_test.items():
            temp.append(v)
        test_feat.append(temp)
    test_feat = np.array(test_feat)
    test_feat_norm = normalize(test_feat, axis=1, norm='l1')
    output = clf.predict(test_feat_norm)
    acc = accuracy_score(truth_test, output, normalize=True)
    loss = hamming_loss(truth_test, output)
    err = [sqrt(mean_squared_error(truth_test[:,i], output[:,i])) for i in range(len(truth_test[0,:]))] 
    accuracy += acc
    #rmse += err
    h_loss += loss
    print("Accuracy for interation " + str(i) + ": " + str(acc))
    for each in err:
        print("RMSE for interation " + str(i) + ": " + str(each))
    print("Hamming loss for interation " + str(i) + ": " + str(loss))
    del test
    del train
    del temp_data

print("Average accuracy is :" + str(accuracy/10))
#print("Average RMSE is :" + str(rmse/10))
print("Average Hamming loss is :" + str(h_loss/10))

Accuracy for interation 0: 0.92
RMSE for interation 0: 0.2
RMSE for interation 0: 0.2449489742783178
RMSE for interation 0: 0.2
RMSE for interation 0: 0.17888543819998318
RMSE for interation 0: 0.1673320053068151
Hamming loss for interation 0: 0.04
Accuracy for interation 1: 0.908
RMSE for interation 1: 0.21908902300206645
RMSE for interation 1: 0.22803508501982758
RMSE for interation 1: 0.20976176963403032
RMSE for interation 1: 0.21908902300206645
RMSE for interation 1: 0.21908902300206645
Hamming loss for interation 1: 0.048
Accuracy for interation 2: 0.916
RMSE for interation 2: 0.18973665961010275
RMSE for interation 2: 0.25298221281347033
RMSE for interation 2: 0.17888543819998318
RMSE for interation 2: 0.17888543819998318
RMSE for interation 2: 0.17888543819998318
Hamming loss for interation 2: 0.0392
